# PYNQ Computer Vision demo: Filter2D and Dilate

This notebook is a modified version of the [filter2d_and_dilate.ipynb](https://github.com/Xilinx/PYNQ-ComputerVision/blob/master/notebooks/computer_vision/filter2d_and_dilate.ipynb) notebook which is part of the [Xilinx PYNQ Computer Vision](https://github.com/Xilinx/PYNQ-ComputerVision) project. Results of this notebook running on the PYNQ-Z1 can be viewed on this [Youtube video](https://www.youtube.com/watch?v=tzQlyEj71Us).

To run this notebook on your PYNQ-Z1:
1. Follow the install instructions for the computer vision project
2. Copy this notebook to your PYNQ-Z1 file system
3. Make the connections described in this [demo video](https://www.youtube.com/watch?v=tzQlyEj71Us)

Read more about the demo on this [FPGA Developer blog post](http://www.fpgadeveloper.com/2018/03/pynq-computer-vision-demo-2d-filter-and-dilate.html).

## Program overlay

Here we program the overlay and load the pynq python libraries for a memory manager and the accelerator drivers.
NOTE: All overlay and python libraries should be loaded prior to assigning the HDMI input/outputs. This is necessary right now to ensure correct functionality but will be enhanced in future releases. For now, please copy this block as is when using it in your own designs.

In [1]:
from pynq.lib.video import *
# from pynq_computervision import BareHDMIOverlay
# base = BareHDMIOverlay("/usr/local/lib/python3.6/dist-packages/pynq_cv/overlays/xv2Filter2DDilate.bit")
from pynq import Xlnk
mem_manager = Xlnk()
# import pynq_computervision.overlays.computer_vision.xv2Filter2DDilate as xv2

hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

NameError: name 'base' is not defined

## Setup and configure HDMI drivers 

~15 seconds to initialize HDMI input/output

In [8]:
hdmi_in.configure(PIXEL_GRAY)
hdmi_out.configure(hdmi_in.mode)

hdmi_in.start()
hdmi_out.start()

## Setup up HDMI input/output parameters 

These parameters are referenced in later function calls

In [9]:
mymode = hdmi_in.mode
print("My mode: "+str(mymode))

height = hdmi_in.mode.height
width = hdmi_in.mode.width
bpp = hdmi_in.mode.bits_per_pixel

str(hdmi_out.mode)

My mode: VideoMode: width=1280 height=720 bpp=8


'VideoMode: width=1280 height=720 bpp=8'

## Run software Filter2D + Dilate

In [10]:
import numpy as np
import cv2

kernel_g = {
        'laplacian high-pass':  np.array([[0.0, 1.0, 0],[1.0, -4, 1.0],[0, 1.0, 0.0]],np.float32),
        'gaussian high-pass': np.array([[-0.0625,-0.125,-0.0625],[-0.125,0.75,-0.125],[-0.0625,-0.125,-0.0625]],np.float32),
        'average blur':  np.ones((3,3),np.float32)/9.0,
        'gaussian blur': np.array([[0.0625,0.125,0.0625],[0.125,0.25,0.125],[0.0625,0.125,0.0625]],np.float32),
        'Sobel Ver': np.array([[1.0,0.0,-1.0],[2.0,0.0,-2.0],[1.0,0.0,-1.0]],np.float32),
        'Sobel Hor': np.array([[1.0,2.0,1.0],[0.0,0.0,0.0],[-1.0,-2.0,-1.0]],np.float32)}

kernelD = np.ones((3,3),np.uint8)
buf     = np.ones((height,width),np.uint8)

numframes = 100

def test_filter(kernelName):
    start = time.time()
    for _ in range(numframes):
        inframe = hdmi_in.readframe()
        outframe = hdmi_out.newframe()
        cv2.filter2D(inframe, -1, kernel_g.get(kernelName, np.ones((3,3),np.float32)/9.0), dst=buf)
        cv2.dilate(buf, kernelD, dst=outframe, iterations=1)
        inframe.freebuffer()
        hdmi_out.writeframe(outframe)
    end = time.time()
    print("Frames per second:  " + str(numframes / (end - start)))

# Test the different filters one by one
test_filter('laplacian high-pass')
test_filter('gaussian high-pass')
test_filter('average blur')
test_filter('gaussian blur')
test_filter('Sobel Ver')
test_filter('Sobel Hor')


Frames per second:  5.319235093125142
Frames per second:  4.503973190394415
Frames per second:  4.627154958597521
Frames per second:  4.627049294217074
Frames per second:  5.090251790582316
Frames per second:  5.158016825525103


## Run hardware accelerated filter2D + dilate 

In [11]:
import numpy as np

kernel_g = {
        'laplacian high-pass':  np.array([[0.0, 1.0, 0],[1.0, -4, 1.0],[0, 1.0, 0.0]],np.float32),
        'gaussian high-pass': np.array([[-0.0625,-0.125,-0.0625],[-0.125,0.75,-0.125],[-0.0625,-0.125,-0.0625]],np.float32),
        'average blur':  np.ones((3,3),np.float32)/9.0,
        'gaussian blur': np.array([[0.0625,0.125,0.0625],[0.125,0.25,0.125],[0.0625,0.125,0.0625]],np.float32),
        'Sobel Ver': np.array([[1.0,0.0,-1.0],[2.0,0.0,-2.0],[1.0,0.0,-1.0]],np.float32),
        'Sobel Hor': np.array([[1.0,2.0,1.0],[0.0,0.0,0.0],[-1.0,-2.0,-1.0]],np.float32)}

def loop_hw2_app(kernelName):
    kernelVoid = np.zeros(0)
    
    xFbuf = mem_manager.cma_array((height,width),np.uint8)
    
    numframes = 1200

    start=time.time()
    for _ in range(numframes):
        inframe = hdmi_in.readframe()
        outframe = hdmi_out.newframe()
        xv2.filter2D(inframe, -1, kernel_g.get(kernelName, np.ones((3,3),np.float32)/9.0), xFbuf, (-1,-1), 0.0,borderType=cv2.BORDER_CONSTANT)
        xv2.dilate(xFbuf, kernelVoid, dst=outframe, borderType=cv2.BORDER_CONSTANT)
        inframe.freebuffer()
        hdmi_out.writeframe(outframe)
    end=time.time()
    print("Frames per second:  " + str(numframes / (end - start)))

# Test the different filters one by one
loop_hw2_app('laplacian high-pass')
loop_hw2_app('gaussian high-pass')
loop_hw2_app('average blur')
loop_hw2_app('gaussian blur')
loop_hw2_app('Sobel Ver')
loop_hw2_app('Sobel Hor')


Frames per second:  60.013129245613996
Frames per second:  60.01214034583174
Frames per second:  60.016618561934905
Frames per second:  60.01684542447948
Frames per second:  60.0161190409898
Frames per second:  60.014465242299245


## Clean up hdmi drivers

NOTE: This is needed to reset the HDMI drivers in a clean state. If this is not run, subsequent executions of this notebook may show visual artifacts on the HDMI out (usually a shifted output image)

In [6]:
hdmi_out.close()
hdmi_in.close()